In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Model imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

# Preprocessing imports
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Set plot style
sns.set(style="whitegrid")

# -------------------------------------------------
# 1. Load Data
# -------------------------------------------------
# We use a direct URL to the 'train.csv' file from the Kaggle competition
data_url = "https_url_to_ames_housing_train.csv" # Placeholder: In a real environment, you'd replace this with the actual URL or local path.
# For a runnable example, let's use a common public version of this dataset:
data_url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv"
# NOTE: The above URL is for the *Boston* housing dataset, which is smaller but will run instantly.
# The logic is identical. Let's proceed with the Boston data for a quick, runnable example.
# If you use the Ames dataset, the principles are the same but the feature names will differ.
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
try:
    data = pd.read_csv(data_url, header=None, sep=',', names=column_names)
    print("Dataset loaded successfully.")

    # Define features (X) and target (y)
    # MEDV (Median value of owner-occupied homes) is our target
    X = data.drop('MEDV', axis=1)
    y = data['MEDV']

except Exception as e:
    print(f"Error loading data: {e}")
    print("Please download the 'train.csv' from the Ames housing link and load it locally.")
    # As a fallback, create dummy data to show the code structure
    # X = pd.DataFrame(np.random.rand(100, 20), columns=[f'feat_{i}' for i in range(20)])
    # X['cat_feat_1'] = np.random.choice(['A', 'B', 'C'], 100)
    # X['cat_feat_2'] = np.random.choice(['X', 'Y'], 100)
    # y = pd.Series(np.random.rand(100) * 1000)

# --- RE-ADAPTATION FOR AMES DATASET (if you load it manually) ---
# If you load the Ames 'train.csv' locally:
# data = pd.read_csv('train.csv')
# X = data.drop(['Id', 'SalePrice'], axis=1)
# # Log-transform the target variable for better performance (it's skewed)
# y = np.log1p(data['SalePrice'])
# -----------------------------------------------------------------


# -------------------------------------------------
# 2. Preprocessing Pipeline
# -------------------------------------------------
# We must scale numerical features and one-hot encode categorical features.
# Regularization is sensitive to the scale of features.

# Identify numerical and categorical features
# (This is a simplified list for the Boston dataset)
numerical_features = X.select_dtypes(include=np.number).columns
categorical_features = X.select_dtypes(include='object').columns

# Create a preprocessing pipeline for numerical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Fill missing values
    ('scaler', StandardScaler())                   # Scale data
])

# Create a preprocessing pipeline for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Fill missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # One-hot encode
])

# Combine pipelines using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# -------------------------------------------------
# 3. Create and Train Models
# -------------------------------------------------

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create full pipelines for each model
# The pipeline will first preprocess the data, then apply the model

# Model 1: Standard Linear Regression (Baseline)
lr_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', LinearRegression())])

# Model 2: Ridge Regression (L2 Regularization)
# alpha is the regularization strength.
ridge_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('model', Ridge(alpha=1.0))])

# Model 3: Lasso Regression (L1 Regularization)
# A higher alpha is often needed for Lasso to show significant feature selection
lasso_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('model', Lasso(alpha=0.1))]) # Start with 0.1 for this dataset

# Train models
print("\nTraining models...")
lr_pipeline.fit(X_train, y_train)
ridge_pipeline.fit(X_train, y_train)
lasso_pipeline.fit(X_train, y_train)
print("Models trained.")

# -------------------------------------------------
# 4. Evaluate Performance
# -------------------------------------------------
models = {
    "Linear Regression": lr_pipeline,
    "Ridge Regression": ridge_pipeline,
    "Lasso Regression": lasso_pipeline
}

print("\n--- Model Performance ---")
results = {}

for name, model in models.items():
    # Make predictions
    y_pred = model.predict(X_test)

    # --- If using Ames data with log-transform ---
    # y_pred = np.expm1(y_pred)
    # y_test_orig = np.expm1(y_test)
    # rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred))
    # r2 = r2_score(y_test_orig, y_pred)
    # ---------------------------------------------

    # --- For Boston data (no log-transform) ---
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    # ---------------------------------------------

    print(f"Model: {name}")
    print(f"  R-squared ($R^2$): {r2:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {rmse:.4f}\n")

    results[name] = {'r2': r2, 'rmse': rmse, 'model_obj': model}


# -------------------------------------------------
# 5. Analyze Regularization Effect (Coefficients)
# -------------------------------------------------
print("\n--- Coefficient Analysis ---")

# Get feature names after preprocessing
# This is tricky but essential for interpretation
try:
    # Ensure preprocessor is fitted for direct access to its components
    preprocessor.fit(X_train)

    # Initialize list for categorical feature names
    cat_features_out = []
    # Only try to get names from OneHotEncoder if there are actual categorical features
    if len(categorical_features) > 0:
        cat_features_out = preprocessor.named_transformers_['cat'] \
                                      .named_steps['onehot'] \
                                      .get_feature_names_out(categorical_features)

    # Combine with numerical feature names
    feature_names = list(numerical_features) + list(cat_features_out)

    # Get coefficients
    lr_coefs = results["Linear Regression"]['model_obj'].named_steps['model'].coef_
    ridge_coefs = results["Ridge Regression"]['model_obj'].named_steps['model'].coef_
    lasso_coefs = results["Lasso Regression"]['model_obj'].named_steps['model'].coef_

    # Create a DataFrame for comparison
    coef_df = pd.DataFrame({
        'Feature': feature_names,
        'LinearReg': lr_coefs,
        'Ridge': ridge_coefs,
        'Lasso': lasso_coefs
    })

    # Analyze the number of zero coefficients
    print(f"Total features after one-hot encoding: {len(feature_names)}")
    print(f"Linear Regression non-zero coefficients: {np.sum(lr_coefs != 0)}")
    print(f"Ridge Regression non-zero coefficients: {np.sum(ridge_coefs != 0)}")
    print(f"Lasso Regression non-zero coefficients: {np.sum(lasso_coefs != 0)}")
    print(f"Lasso eliminated {np.sum(lasso_coefs == 0)} features.")

    # Plot coefficient magnitudes
    plt.figure(figsize=(15, 6))

    # Plot Lasso coefficients
    plt.subplot(1, 2, 1)
    lasso_coefs_sorted = pd.Series(lasso_coefs, index=feature_names).sort_values()
    lasso_coefs_sorted[lasso_coefs_sorted != 0].plot(kind='barh', title='Lasso Coefficients (L1)')

    # Plot Ridge coefficients
    plt.subplot(1, 2, 2)
    ridge_coefs_sorted = pd.Series(ridge_coefs, index=feature_names).sort_values()
    ridge_coefs_sorted.plot(kind='barh', title='Ridge Coefficients (L2)', color='skyblue')

    plt.suptitle("Effect of Regularization on Model Coefficients")
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

except Exception as e:
    print(f"\nCould not generate coefficient plot: {e}")
    print("This step requires the model to be trained successfully on data with categorical features.")

Dataset loaded successfully.

Training models...
Models trained.

--- Model Performance ---
Model: Linear Regression
  R-squared ($R^2$): 0.6688
  Root Mean Squared Error (RMSE): 4.9286

Model: Ridge Regression
  R-squared ($R^2$): 0.6685
  Root Mean Squared Error (RMSE): 4.9308

Model: Lasso Regression
  R-squared ($R^2$): 0.6501
  Root Mean Squared Error (RMSE): 5.0652


--- Coefficient Analysis ---

Could not generate coefficient plot: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.
This step requires the model to be trained successfully on data with categorical features.
